# Transformers From Scratch
You've heard the name, you've seen the papers, you've probably even stepped through a few repos. But can you build a multi-head attention mechanism from scratch?

In this lab you'll be asked to implement the novel multi-head attention function from scratch, and plug this into our provided framework to test that your function works.

We'll also ask you to parralellize it to take advantage of all the cores in your machine. 

So hang in there, and let's get to it!

* Note the computational and data needs of this lab are pretty intensive. I was able to run successfully on a notebook instance using ml.p3.16xlarge. If you don't already have that instance you might want to cut a ticket for it.

---
# Task 0 - Learn About This Script Pacakge
First, let's examine the codebase here. You'll see a Transformer-based model architecture, so to speak, implemented here. The catch? There is no multi-head attention function. 

We'll step through the codebase with you here in the guide so you know the key points. At the end we'll run a quick functional test to make sure you can indeed run a basic training job.

Task 0 should take ~ 10 minutes. Try to take your time reading through this so you know what the arguments are and how the script is constructed.

In [23]:
# this package is a fork of Peter Bloem's "former" repository, where he implements a Transformer from scratch in PyTorch.
!git clone https://github.com/EmilyWebber/former.git

First, this codebase supports two modes. One for text classification, the other for text generation. Both of those can be run by a single Python script, `python experiments/classify.py`. You'll see that we are just calling that Python script inside of the `model.py` script that SageMaker uses to execute the training job.

Inside of `classify.py`, you'll see we're creating a model on line 60. `model = former.CTransformer`. Note that this model takes a few hyperparameters - `embedding_size`, `num_heads`, `depth`, `seq_length`, `num_classes`, and `max_pool`. 

![](images/model_create.png)

Now let's check out that `CTransformer()` object. You'll see it's inherited from the `former` class. 

Inside `transformers.py`, there's an `__init__` for the `CTransformer()` object. Inside the init, we'll see a small for-loop defined that pulls in the hyperparameters we just passed in, and creates a `TransformerBlock`. Then, it's using the PyTorch `nn.Sequential` API to convert those blocks into a sequential neural network component.

![](images/TransformerBlock.png)

Now, where is this `TransformerBlock` defined? It's actually in `modules.py`.  You'll see one `TransformerBlock` class, that inherits a `SelfAttentionWide` or `SelfAttentionNarrow` object on creation. We'll follow the rabbit-hole on the `SelfAttentionNarrow.`

The `SelfAttentionWide` init defines a few objects: `self.tokeys`, `self.toqueries`, and `self.tovalues`.

![](images/tokeys.png)

Then, we see a `forward` function that calls `self.tokeys`. Next, it computes a scaled dot-product self-attention function! That's what we need to implement. 

![](images/self-attention.png)

Sound like fun? Now, run your base training job to make sure the pipes are working today. 

In [7]:
# !pip install --upgrade sagemaker

In [24]:
# run the model file
import sagemaker
import os

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

prefix = 'transformers'

role = sagemaker.get_execution_role()

# create some arbitrary train file that we won't use
!echo 1,2,3,4 > holder_file.csv 
s3_train_path = "s3://{}/{}/train/{}".format(bucket, prefix, 'holder_file.csv')
os.system('aws s3 cp {} {}'.format( 'holder_file.csv', s3_train_path))

from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='model.py',
                    role=role,
                    framework_version='1.2.0',
                    py_version = 'py3',
                    source_dir = 'former',
                    instance_count=1,
                    instance_type = 'ml.p3.2xlarge')

estimator.fit({'training': s3_train_path}, wait=False)

---
# Task 1 - Implement a Multi-head Attention Function
Now, here's the fun part. Can you implement your own multi-head attention mechanism? Don't worry, we'll give you all the tips you need.

Task 1 should take ~30 minutes.

In [10]:
!pip install -r former/requirements.txt

As you're stepping through the code below to build your neural network, don't panic. You can actually look at the pictures provided in this notebook here, along with the code from the `former` repo, for inspiration. 

In [33]:
%%writefile former/former/my_transformer_model.py
from torch import nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import math
from torchtext import data, datasets, vocab

import numpy as np

def my_multihead_attention_mechanism(x, emb = 128, h = 8):
    
    b, t, e = # your code here
    assert e == emb, f'Input embedding dim ({e}) should match layer embedding dim ({self.emb})'

    s = # your code here 
    x = # your code here 

    tokeys = # your code here 
    toqueries = # your code here 
    tovalues  = # your code here 

    keys = # your code here 
    queries = # your code here 
    values  = # your code here 
    
    assert keys.size() == (b, t, h, s)
    assert queries.size() == (b, t, h, s)
    assert values.size() == (b, t, h, s)
    
    # compute scaled dot-product self-attention

    # - fold heads into the batch dimension
    keys = # your code here 
    queries = # your code here 
    values = # your code here 

    queries = # your code here 
    keys    = # your code here 
    # - Instead of dividing the dot products by sqrt(e), we scale the keys and values.
    #   This should be more memory efficient

    # - get dot product of queries and keys, and scale
    dot = # your code here 

    assert dot.size() == (b*h, t, t)

    if self.mask: # mask out the upper half of the dot matrix, excluding the diagonal
        mask_(dot, maskval=float('-inf'), mask_diagonal=False)

    dot = # your code here 
    # - dot now has row-wise self-attention probabilities

    # apply the self attention to the values
    out = # your code here 

    # swap h, t back, unify heads
    out = # your code here 

    unifyheads = # your code here 

    return unifyheads(out)


Overwriting former/former/my_transformer_model.py


It turns out this package is just happier being run as a script - when you run this line it's pointing to a string of custom objects that will ultimately look for your `my_attention_mechanism` function as the base of the model.

In [30]:
!python former/experiments/custom_classify.py

OPTIONS  Namespace(batch_size=4, depth=6, embedding_size=128, final=False, gradient_clipping=1.0, lr=0.0001, lr_warmup=10000, max_length=512, max_pool=False, num_epochs=80, num_heads=8, seed=1, tb_dir='./runs', vocab_size=50000)
- nr. of training examples 5000
- nr. of validation examples 1250

 epoch 0
  0%|                                                  | 0/5000 [00:00<?, ?it/s]looping through batches
got our first input!
  0%|                                                  | 0/5000 [00:00<?, ?it/s]


You may be thinking, hey, I thought we were going to train a full model here? And the sad reality is that, at 11pm the night before the workshop, I simply ran out of steam to continue debugging the error statements on the custom classifier. If you are interested in and willing it give it a shot, please by all means do so. Otherwise, on we go!

Now, let's test that out on SageMaker! 

In [31]:
%%writefile former/my_new_model.py

import os

if __name__ == '__main__':
    os.system('python experiments/custom_classify.py')

Overwriting former/my_new_model.py


In [32]:
# run the model file
import sagemaker
import os

sagemaker_session = sagemaker.Session()

from sagemaker.pytorch import PyTorch

role = sagemaker.get_execution_role()

estimator = PyTorch(entry_point='my_new_model.py',
                    role=role,
                    framework_version='1.2.0',
                    py_version = 'py3',
                    source_dir = 'former',
                    instance_count=1,
                    instance_type = 'ml.p3.2xlarge')

estimator.fit({'training': s3_train_path}, wait=False)

---
# Task 2 - Compare your Results
When we have about 30 minutes left, the MC team will share the solution notebook with you so you can compare notes. Should take only a few minutes if your solution is working, otherwise you may want to step through it a bit more carefully.

And that's a wrap! If you made it here with extra time to spare, consider doubling back on some of those functions and try to really understand what's going on there. 

And that's a wrap! If you made it here with extra time to spare, consider doubling back on some of those functions and try to really understand what's going on there. 